In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cd /content/drive/MyDrive/6389_Assignment2/6389Project2Data

In [ ]:
import pandas as pd
import numpy as np
import torch

In [ ]:
!pip install torch torchvision

In [ ]:
!pip install torch_geometric
!pip install torch_sparse
!pip install torch_scatter

In [ ]:
# import pandas as pd
# import numpy as np
# import torch
# # LOAD dataset
# from google.colab import drive
# drive.mount('/content/drive')
# import numpy as np
# func_data = [ np.loadtxt(f"/content/drive/MyDrive/6389_Assignment2/6389Project2Data/{n}{i}/FunctionalConnectivity.txt") for n in ["AD","CN"] for i in range(1,11)]
# struct_data = [ np.loadtxt(f"/content/drive/MyDrive/6389_Assignment2/6389Project2Data/{n}{i}/StructuralConnectivity.txt") for n in ["AD","CN"] for i in range(1,11)]
# labels = [ int(n == "AD") for n in ["AD","CN"] for i in range(1,11)]

# func_data = torch.tensor(np.array(func_data), dtype=torch.float32)
# struct_data = torch.tensor(np.array(struct_data), dtype=torch.float32)
# labels = torch.tensor(np.array(labels), dtype=torch.float32)

# from sklearn.model_selection import train_test_split
# functional_data_train, functional_data_test, labels_train, labels_test = train_test_split(
#     func_data, labels, test_size=0.2, random_state=42)
# structural_data_train, structural_data_test, _, _ = train_test_split(
#     struct_data, labels, test_size=0.2, random_state=42)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader
import numpy as np

class GNNModel(nn.Module):
    def __init__(self):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(300, 64)  # Input features: 2 (identity matrix), Output features: 16
        self.conv2 = GCNConv(64, 16)  # Output features: 1
        self.fc = nn.Linear(2400, 2)  # Final fully connected layer for batch_size x 2 label size

    def forward(self, x):
        num_nodes = 150

# Generate fully connected graph edge indices
        edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long).t().contiguous()
        edge_index=edge_index.to('cpu')
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        x = x.reshape(2,2400)
        x=self.fc(x)
        return x

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision.datasets import Food101
import os
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.image as image
#import matplotlib.pyplot as plt
from torch_geometric.utils import dense_to_sparse


class Dataset():
   def __init__(self,root,split,transform=None):

      self.files_adresses_labels=[]
      self.transform=transform
      self.split=split

      for subfolder in os.listdir(root):
               if subfolder[:2] =='AD':
                    for file in os.listdir(os.path.join(root,subfolder)):
                        self.files_adresses_labels.append(((os.path.join(root,subfolder,file).replace("\\","/")),1))
               else:
                    for file in os.listdir(os.path.join(root,subfolder)):
                        self.files_adresses_labels.append(((os.path.join(root,subfolder,file).replace("\\","/")),0))
      self.files_adresses_labels= [[self.files_adresses_labels[i], self.files_adresses_labels[i + 1]] for i in range(0, len(self.files_adresses_labels), 2)]
      self.train_files_adresses_labels,self.test_files_adresses_labels=  train_test_split(self.files_adresses_labels, test_size=0.2, random_state=42)

   def __len__(self):
      if self.split=='train':

       return len(self.train_files_adresses_labels)

      elif self.split=='test':
       return len(self.test_files_adresses_labels)

   def __getitem__(self,index):


      if self.split=='train':

         functional_connectivity = np.loadtxt(self.train_files_adresses_labels[index][0][0])
         structural_connectivity = np.loadtxt(self.train_files_adresses_labels[index][1][0])
         data = torch.tensor(np.concatenate([functional_connectivity, structural_connectivity], axis=1), dtype=torch.float32)
         #adj=functional_connectivity

         num_nodes = 150

         edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long).t().contiguous()

         label= (self.train_files_adresses_labels[index][1][1])


      if self.split=='test':

         functional_connectivity = np.loadtxt(self.test_files_adresses_labels[index][0][0])
         structural_connectivity = np.loadtxt(self.test_files_adresses_labels[index][1][0])
         data= torch.tensor(np.concatenate([functional_connectivity, structural_connectivity], axis=1), dtype=torch.float32)

         num_nodes = 150

# Generate fully connected graph edge indices
         edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long).t().contiguous()

         label= (self.test_files_adresses_labels[index][1][1])

      if self.transform is not None:

        img=self.transform(img)
      return data, label

In [ ]:
# from model import GNNModel
# from dataset import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision.datasets import Food101
import os
import numpy as np
import matplotlib.image as image

root='/content/drive/MyDrive/6389_Assignment2/6389Project2Data/'


train_dataset=Dataset(root,split='train')
test_dataset=Dataset(root,split='test')


train_loader=DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader=DataLoader(test_dataset, batch_size=2, shuffle=True)


#######################################Hyperparameters######################################

learning_rate=0.001
device='cpu'
number_of_epochs=15


#####################################initializing network################################
model = GNNModel()

model=model.to(device)

Loss=nn.CrossEntropyLoss()

optimizer=optim.Adam(model.parameters(), lr=learning_rate)


train_accuracy=[]
train_Loss=[]
#val_accuracy=[]
#val_Loss=[]

for epoch in range(number_of_epochs):
    num_correct=0
    num_tot=0
    L=0

    #################training


    for Input, label in train_loader:

        label=label.tolist()
        #labels_batch = [0, 1]  # Replace this with your batch of labels
        num_classes = 2  # Number of unique classes (0 and 1)


        one_hot_encodings = np.eye(num_classes)[label]

        one_hot_encodings= torch.tensor(one_hot_encodings)

        Input=Input.float()

        Input=Input.to(device)

        one_hot_encodings= one_hot_encodings.float()
        one_hot_encodings= one_hot_encodings.to(device)

        output=model(Input)

        loss=Loss(output, one_hot_encodings)
        #print('""""""""""""""""""""""""')
        #print('loss', loss)


        L=loss.item()+L




        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


# Convert one-hot labels to class labels

        predicted_classes = torch.argmax(output, axis=1)
        num_correct= torch.sum(predicted_classes == torch.argmax(one_hot_encodings, axis=1))+num_correct

        num_tot= one_hot_encodings.size(0)+num_tot

    train_acc=float(num_correct)/float(num_tot)

    train_accuracy.append(train_acc)
    L=L/len(train_loader)
    train_Loss.append(L)

def test_accuracy_loss(model, test_loader):

    test_accuracy=[]
    test_Loss=[]

    y_pred=[]
    y_true=[]
    num_tot=0
    num_correct=0
    test_loss=0
    L=0

    model.eval()
    with torch.no_grad():

     print('""""""""""""""')

     for Input, label in test_loader:
        label=label.tolist()
        #labels_batch = [0, 1]  # Replace this with your batch of labels
        num_classes = 2  # Number of unique classes (0 and 1)

        one_hot_encodings = np.eye(num_classes)[label]

        one_hot_encodings= torch.tensor(one_hot_encodings)

        Input=Input.float()

        Input=Input.to(device)

        one_hot_encodings= one_hot_encodings.float()
        one_hot_encodings= one_hot_encodings.to(device)

        output=model(Input)

        loss=Loss(output, one_hot_encodings)

        L=loss.item()+L

        predicted_classes = torch.argmax(output, axis=1)
        num_correct= torch.sum(predicted_classes == torch.argmax(one_hot_encodings, axis=1))+num_correct

        num_tot= one_hot_encodings.size(0)+num_tot

    test_acc=float(num_correct)/float(num_tot)

    test_accuracy.append(test_acc)
    L=L/len(test_loader)
    test_Loss.append(L)

    return test_Loss, test_accuracy

test_acc,test_loss=test_accuracy_loss(model, test_loader)

print('train_acc= ', train_accuracy)
print('train_Loss= ', train_Loss)
#print('val_accuracy= ', val_accuracy)
#print('val_Loss= ', val_Loss)
print('test_acc= ', test_acc)
print('test_loss= ', test_loss)

""""""""""""""
train_acc=  [0.3125, 0.6875, 0.8125, 0.8125, 0.8125, 0.8125, 0.75, 0.9375, 0.9375, 0.9375, 1.0, 1.0, 1.0, 1.0, 1.0]
train_Loss=  [24.33430819644127, 1.472511356434552, 0.9403998714811905, 0.9036345324505533, 1.538659080852248, 6.440930874994706, 2.259920280297589, 0.22546506582511583, 0.1001745752143961, 0.25864959504355056, 0.0, 0.0, 1.043080430918053e-07, 1.4901144140821998e-07, 2.45868676529426e-07]
test_acc=  [7.500840798020363]
test_loss=  [0.5]
